# 1. Import Library yang kita butuhkan

- Pada perintah di bawah kami mengimport semua kebutuhan library yang kami butuhkan,
- Jika temen temen tidak mau import semua library yang di butuhkan dalam step 1 temen temen cukup import library json saja untuk menghubungkan dataset yang sudah kita buat.

### Perlu di note sebelumnya di virtual environment kami sudah Install and import library yang di butuhkan seperti **tensorflow, keras, keras-models, pickle, nltk**

Cara install:

- pip install tensorflow
- pip install keras
- pip install keras-models
- pip install pickle
- pip install nltk



In [6]:
# import necessary libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle

import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random
from keras.models import load_model

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# importing the GL Bot corpus file for pre-processing

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open("data.json").read()
intents = json.loads(data_file)

# 2. Data pre-processing

In [7]:
# preprocessing the json data
# tokenization
#nltk.download('punkt')
#nltk.download('wordnet')
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

### Tokenisasi

- Pada proses tokenisasi pada dasarnya adalah pemisahan kalimat, paragraf, 
atau seluruh dokumen teks menjadi unit yang lebih kecil, proses itu yang disebut token

- Pada proses ini juga akan save documen tersebut menjadi file label.pkl dan texts.pkl (proses labeling)

In [8]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('texts.pkl','wb')) 
pickle.dump(classes,open('labels.pkl','wb'))

117 documents
115 classes ['greeting1', 'greeting10', 'greeting11', 'greeting12', 'greeting13', 'greeting14', 'greeting15', 'greeting16', 'greeting2', 'greeting3', 'greeting4', 'greeting5', 'greeting6', 'greeting7', 'greeting8', 'greeting9', 'greeting_malam', 'noanswer', 'pengalaman1', 'pengalaman2', 'pengalaman3', 'pengalaman4', 'pengalaman5', 'pengalaman6', 'pengalaman7', 'pengalaman8', 'pengalaman9', 'penutup1', 'penutup1a', 'penutup2', 'penutup3', 'penutup4', 'penutup5', 'skill1', 'skill10', 'skill11', 'skill12', 'skill13', 'skill14', 'skill15', 'skill16', 'skill17', 'skill18', 'skill19', 'skill2', 'skill20', 'skill21', 'skill22', 'skill23', 'skill24', 'skill25', 'skill26', 'skill27', 'skill28', 'skill29', 'skill3', 'skill30', 'skill31', 'skill32', 'skill33', 'skill34', 'skill35', 'skill36', 'skill37', 'skill38', 'skill39', 'skill4', 'skill40', 'skill41', 'skill42', 'skill43', 'skill44', 'skill45', 'skill46', 'skill47', 'skill48', 'skill49', 'skill5', 'skill50', 'skill51', 'skill52

# 3. Creating Training Data

- Pada dasarnya, bag of words adalah representasi sederhana dari setiap teks dalam sebuah kalimat sebagai bag of words-nya.

In [9]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
   
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])

print("Training data created")

Training data created


# 5. Creating Modeling 

- Pada proses ini kami akan membuat model jaringan saraf dan menyimpan model tersebut 

In [10]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('models.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")

Epoch 1/200
24/24 [==============================] - 0s 1ms/step - loss: 4.7676 - accuracy: 0.0000e+00
Epoch 2/200
24/24 [==============================] - 0s 1ms/step - loss: 4.7394 - accuracy: 0.0342
Epoch 3/200
24/24 [==============================] - 0s 1ms/step - loss: 4.7237 - accuracy: 0.0085
Epoch 4/200
24/24 [==============================] - 0s 1ms/step - loss: 4.7185 - accuracy: 0.0171
Epoch 5/200
24/24 [==============================] - 0s 1ms/step - loss: 4.7171 - accuracy: 0.0256
Epoch 6/200
24/24 [==============================] - 0s 1ms/step - loss: 4.7018 - accuracy: 0.0256
Epoch 7/200
24/24 [==============================] - 0s 1ms/step - loss: 4.6950 - accuracy: 0.0171
Epoch 8/200
24/24 [==============================] - 0s 2ms/step - loss: 4.6650 - accuracy: 0.0342
Epoch 9/200
24/24 [==============================] - 0s 1ms/step - loss: 4.6521 - accuracy: 0.0171
Epoch 10/200
24/24 [==============================] - 0s 2ms/step - loss: 4.6297 - accuracy: 0.0598
Epoch